In [1]:

# @hidden_cell
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_d3bd5b94a9334de59a55a7fed2bedeaa(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage V3 using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '6aaf54352357483486ee2d4981f8ef15')
    hconf.set(prefix + '.username', '1c3a895fa0d04923aec623a25013e8ed')
    hconf.set(prefix + '.password', 'i33i[1QlOBJ{jvSL')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', True)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_d3bd5b94a9334de59a55a7fed2bedeaa(name)

df_data_2 = sqlContext.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true')\
    .load("swift://coursera2." + name + "/PAIRS_DATA_11_19_16T18_45_37.csv")
df_data_2.take(5)


[Row(Dataset=u'Daymet', Datalayer=u'Daily maximum temperature', Timestamp=u'12/31/14T00:00:00', Lat=35.15184020996094, Lon=-115.97535705566406, Units=u'C', Value=10.517, DaymetDailymaximumtemperature=10.517, DaymetDailyminimumtemperature=None, DaymetPrecipitationrate=None, DaymetShortwaveradiationdailymean)=None, DaymetVaporpressuredailymean=None),
 Row(Dataset=u'Daymet', Datalayer=u'Daily maximum temperature', Timestamp=u'12/30/14T00:00:00', Lat=35.15184020996094, Lon=-115.97535705566406, Units=u'C', Value=12.053, DaymetDailymaximumtemperature=12.053, DaymetDailyminimumtemperature=None, DaymetPrecipitationrate=None, DaymetShortwaveradiationdailymean)=None, DaymetVaporpressuredailymean=None),
 Row(Dataset=u'Daymet', Datalayer=u'Daily maximum temperature', Timestamp=u'12/29/14T00:00:00', Lat=35.15184020996094, Lon=-115.97535705566406, Units=u'C', Value=11.499, DaymetDailymaximumtemperature=11.499, DaymetDailyminimumtemperature=None, DaymetPrecipitationrate=None, DaymetShortwaveradiation

In [2]:
df_data_2.printSchema()

root
 |-- Dataset: string (nullable = true)
 |-- Datalayer: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Units: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- DaymetDailymaximumtemperature: double (nullable = true)
 |-- DaymetDailyminimumtemperature: double (nullable = true)
 |-- DaymetPrecipitationrate: double (nullable = true)
 |-- DaymetShortwaveradiationdailymean): double (nullable = true)
 |-- DaymetVaporpressuredailymean: double (nullable = true)



In [31]:
df_data_2.createOrReplaceTempView("df")
aggregates = sqlContext.sql("""
select 
    min(DaymetDailymaximumtemperature),max(DaymetDailymaximumtemperature),mean(DaymetDailymaximumtemperature),
    min(DaymetDailyminimumtemperature),max(DaymetDailyminimumtemperature),mean(DaymetDailyminimumtemperature),
    min(DaymetPrecipitationrate),max(DaymetPrecipitationrate),mean(DaymetPrecipitationrate) as meanDPR,
    min(DaymetVaporpressuredailymean),max(DaymetVaporpressuredailymean),mean(DaymetVaporpressuredailymean),
    cw,
    cast (concat('20',year) as int) as year
from (
    select 
        weekofyear(to_date(concat('20',split(split(Timestamp,'T')[0],'/')[2],'-',split(split(Timestamp,'T')[0],'/')[0],'-',split(split(Timestamp,'T')[0],'/')[1],' 00:00:00'))) as cw,
        split(split(Timestamp,'T')[0],'/')[2] as year,
        DaymetDailymaximumtemperature,
        DaymetDailyminimumtemperature,
        DaymetPrecipitationrate,
        DaymetVaporpressuredailymean
        from df
) group by cw,year
""")


In [32]:
from pyspark.sql.types import *
df_data_3 = sqlContext.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true')\
    .load("swift://coursera2." + name + "/final.csv")
df_data_3.take(5)


[Row(year=u'1980', CommodityCode=u'268099', CropName=u'ALMOND HULLS ', CountyCode=u'7', County=u'Butte ', HarvestedAcres=u'  ', Yield=u'  ', Production=u'30000', Pricepu=u'59', Unit=u'TONS ', Value=u'1770000'),
 Row(year=u'1980', CommodityCode=u'268099', CropName=u'ALMOND HULLS ', CountyCode=u'11', County=u'Colusa ', HarvestedAcres=u'  ', Yield=u'  ', Production=u'5250', Pricepu=u'60', Unit=u'TONS ', Value=u'315000'),
 Row(year=u'1980', CommodityCode=u'268099', CropName=u'ALMOND HULLS ', CountyCode=u'19', County=u'Fresno ', HarvestedAcres=u'  ', Yield=u'  ', Production=u'23700', Pricepu=u'75', Unit=u'TONS ', Value=u'1778000'),
 Row(year=u'1980', CommodityCode=u'268099', CropName=u'ALMOND HULLS ', CountyCode=u'29', County=u'Kern ', HarvestedAcres=u'  ', Yield=u'  ', Production=u'88600', Pricepu=u'77', Unit=u'TONS ', Value=u'6811000'),
 Row(year=u'1980', CommodityCode=u'268099', CropName=u'ALMOND HULLS ', CountyCode=u'31', County=u'Kings ', HarvestedAcres=u'  ', Yield=u'  ', Production=u

In [33]:
joint = aggregates.join(df_data_3,aggregates.year==df_data_3.year)
joint2 = joint.filter(joint.CountyCode==999).filter(joint.CropName=='WOOL')\
    .select('meanDPR','Production')\
    .withColumn("ProductionTmp", df_data_3.Production.cast(IntegerType()))\
    .drop("Production")\
    .withColumnRenamed("ProductionTmp", "Production")

In [6]:
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.feature import VectorAssembler

#assembler = VectorAssembler(
#    inputCols=["year", "Production" ],
#    outputCol="features")
#output = assembler.transform(joint2)


In [34]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

train = joint2.rdd.map(lambda x:LabeledPoint(int(x.Production), [int(x.meanDPR)]))


In [35]:
model = LinearRegressionWithSGD.train(train, iterations=100, step=0.00000001)

In [36]:
# Evaluate the model on training data
valuesAndPreds = train.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds \
    .map(lambda (v, p): (v - p)**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 5.03402929344e+12
